// todo 
// examples for falsy and truthy values
// examples for ?? 
// examples for !!

This notebook demonstrates simple asynchronous patterns in JavaScript and a tiny EventEmitter implementation (ES5). It mirrors the examples from the repository files and groups them into explanatory sections.
Follow the Table of Contents: falsy/truthy, ??, !!, var vs let hoisting, event loop notes, EventEmitter (ES5), Node EventEmitter usage, promises.

## Table of Contents
1. Falsy and Truthy Values
2. Nullish coalescing `??`
3. Double negation `!!`
4. Variable hoisting: `var` vs `let` in setTimeout loops
5. Event loop notes (`setTimeout`, `setImmediate`)
6. Building EventEmitter from scratch (ES5)
7. Node's `EventEmitter` usage (example from repository)
8. Promises example
9. Summary & next steps

### 1 — Falsy and Truthy Values
Common falsy values in JS: `false`, `0`, `''` (empty string), `null`, `undefined`, `NaN`. Everything else is truthy.
Small examples below show how values behave in conditionals and when coerced to boolean.

In [ ]:
// Falsy and truthy examples
const vals = [false, 0, '', null, undefined, NaN, '0', 'hello', 1, {}];
vals.forEach(v => console.log(JSON.stringify(v), '->', Boolean(v)));

// Example: conditional behavior
if (0) { console.log('0 is truthy'); } else { console.log('0 is falsy'); }
if ('hello') { console.log('hello is truthy'); } else { console.log('hello is falsy'); }

### 2 — Nullish coalescing (`??`)
The `??` operator returns the right-hand side when the left-hand side is `null` or `undefined` (but not other falsy values like `0` or `''`).

In [ ]:
console.log(null ?? 'default'); // 'default'
console.log(undefined ?? 'default'); // 'default'
console.log(0 ?? 'default'); // 0 (not nullish)
console.log('' ?? 'default'); // '' (not nullish)
// Compare with ||
console.log(0 || 'fallback'); // 'fallback' because 0 is falsy
console.log(null || 'fallback'); // 'fallback'

### 3 — Double negation (`!!`)
`!!value` converts a value to its boolean equivalent. Useful for explicit truthiness checks.

In [ ]:
console.log(!!0); // false
console.log(!!''); // false
console.log(!!'hello'); // true
console.log(!!{}); // true

### 4 — Variable hoisting: `var` vs `let` in loops
This classic example shows how `var` is function-scoped and causes closure issues inside asynchronous callbacks, while `let` is block-scoped and avoids the problem.

In [ ]:
console.log('var loop:');
for (var i = 0; i < 3; i++) {
  setTimeout(() => {
    console.log('var i ->', i);
  }, 50);
}

console.log('let loop:');
for (let j = 0; j < 3; j++) {
  setTimeout(() => {
    console.log('let j ->', j);
  }, 100);
}

### 5 — Event loop notes (`setTimeout` vs `setImmediate`)
- `setTimeout(fn, 0)` schedules a macrotask after the current call stack and any microtasks.
- `setImmediate` (Node) schedules a callback to run on the check phase — typically runs after I/O events, and in many cases appears similar to `setTimeout(fn, 0)`, but ordering can differ.
Small demos below use both patterns.

In [ ]:
console.log('start');
setTimeout(() => console.log('timeout 0'), 0);
if (typeof setImmediate === 'function') {
  setImmediate(() => console.log('setImmediate'));
} else {
  console.log('setImmediate not available in this environment');
}
Promise.resolve().then(() => console.log('microtask (promise)'));
console.log('end');

### 6 — Building EventEmitter from scratch (ES5)
Below is the simple ES5 EventEmitter implementation from `event.emit.test.js`. It demonstrates `on()` and `emit()` and uses `setTimeout` inside `emit()` to invoke listeners asynchronously.
(This is kept small and educational — not a production replacement.)

In [ ]:
function EventEmitter() {
  this.events = {};
}

EventEmitter.prototype.on = function(eventName, fn) {
  if (!this.events[eventName]) {
    this.events[eventName] = [];
  }
  this.events[eventName].push(fn);
};

EventEmitter.prototype.off = function(eventName, fn) {
  var listeners = this.events[eventName];
  if (!listeners) return;
  for (var i = listeners.length - 1; i >= 0; i--) {
    if (listeners[i] === fn) {
      listeners.splice(i, 1);
    }
  }
};

EventEmitter.prototype.once = function(eventName, fn) {
  var self = this;
  function wrapper() {
    // call the original listener with the provided args
    fn.apply(null, arguments);
    // remove the wrapper after first invocation
    self.off(eventName, wrapper);
  }
  this.on(eventName, wrapper);
};

EventEmitter.prototype.emit = function(eventName) {
  var args = Array.prototype.slice.call(arguments, 1);
  var event = this.events[eventName];
  if (event) {
    event.forEach(function(fn) {
      setTimeout(function() {
        fn.apply(null, args);
      }, 0);
    });
  }
};

// Usage demo
var emitter = new EventEmitter();
console.log('setup the events');

function listenerA() { console.log('listenerA called'); }
function listenerB() { console.log('listenerB called'); }

// add two listeners and then remove one
emitter.on('test', listenerA);
emitter.on('test', listenerB);
console.log('emit 1 (both should run):');
emitter.emit('test');

// remove listenerA before next emit
emitter.off('test', listenerA);
console.log('emit 2 (only listenerB should run):');
emitter.emit('test');

// once demo
emitter.once('onceEvent', function() { console.log('onceEvent fired'); });
console.log('emit onceEvent first time:');
emitter.emit('onceEvent');
console.log('emit onceEvent second time (should not fire):');
emitter.emit('onceEvent');

// Use setImmediate if available (Node). Otherwise, emit after a short timeout for initial demo scheduling.
if (typeof setImmediate === 'function') {
  setImmediate(function() { /* already emitted above */ });
} else {
  // nothing extra needed; emits were scheduled with setTimeout inside emit()
}

console.log('after demo emits');

### 7 — Node's `EventEmitter` usage (from repository `event.emit.js`)
Example shows using Node's built-in `events` module and emitting events with arguments. If running this notebook in a Node-backed JS kernel, you can `require('events')`. Otherwise, read as demonstration.

In [ ]:
// Node example (requires a Node-backed kernel).
try {
  const EventEmitter = require('events');
  const em = new EventEmitter();
  em.on('greet', function(name) {
    console.log(`Hello, ${name}!`);
  });
  em.on('user', function(id, name, age) {
    console.log(`User ${id}: ${name}, ${age} years old`);
  });
  // emit some events after a delay
  setTimeout(() => {
    em.emit('greet', 'Alice');
    em.emit('user', 123, 'Bob', 25);
  }, 500);
} catch (err) {
  console.log('Node built-in EventEmitter example skipped (not available in this kernel).');
}

### 8 — Promises example
A small promise demo copied from `event.emit.test.js`. It resolves after 5 seconds in the original file; here we shorten the delay for an interactive notebook.

In [ ]:
let p = new Promise((resolve, reject) => {
  setTimeout(() => {
    if (false) {
      reject('Promise rejected!');
      return;
    }
    resolve('Promise resolved!');
  }, 200); // shortened for demo
});

p.then(message => console.log(message));
p.then(message => console.log('Second then: ' + message));

### 9 — Summary & Next Steps
- You saw falsy/truthy, `??`, `!!`, var vs let hoisting, event loop ordering, a small ES5 EventEmitter, Node EventEmitter usage, and a Promise example.
- Next steps (optional): add `off()` to the ES5 emitter, demonstrate error handling in emit, add short exercises, or convert the ES5 class to ES6 `class` syntax.